In [2]:
# 示例代码
# from xgboost import XGBClassifier
# # read data
# from sklearn.datasets import load_iris
# from sklearn.model_selection import train_test_split
# data = load_iris()
# X_train, X_test, y_train, y_test = train_test_split(data['data'], data['target'], test_size=.2)
# # create model instance
# bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
# # fit model
# bst.fit(X_train, y_train)
# # make predictions
# preds = bst.predict(X_test)

In [3]:
import xgboost as xgb
from xgboost import plot_importance
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from pandas import read_csv
import numpy as np
from xgboost.sklearn import XGBClassifier, XGBRegressor
from sklearn import metrics
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import xgboost as xgb
from xgboost import XGBClassifier, plot_importance
import warnings

In [4]:
df = read_csv("2024c/traintestds/2023-wimbledon-1401", index_col=[0])
df.head()

,score_dif_p1,score_dif_p2,server_p1,server_p2,p1_ad,p2_ad,p1_comble,p2_comble,p1_break_points,p2_break_points,p1_break_points_won,p2_break_points_won,p1_win_possiblity,p2_win_possiblity
3788,-0.810852,0.810852,0,1,0,0,-0.420574,-0.381385,0,0,0,0,3,2
3789,-0.638412,0.638412,0,1,0,0,-0.420574,-0.381385,0,0,0,0,2,3
3790,-0.810852,0.810852,0,1,0,0,1.510632,-0.381385,0,0,0,0,1,4
3791,-0.638412,0.638412,0,1,0,0,-0.420574,-0.381385,0,0,0,0,1,4
3792,-0.810852,0.810852,0,1,0,0,-0.420574,-0.381385,1,0,0,0,0,5


In [5]:
def ds_concat(dir_path='2024c/traintestds/', df_all_in_one_pth=None):
    if df_all_in_one_pth != None:
        df_list = [read_csv(df_all_in_one_pth,index_col=[0])]
    else:
        df_list = [read_csv(dir_path+fn,index_col=[0]) for fn in os.listdir(dir_path)]
    x_list, y_list = [], []
    for df in df_list:
        df = df.drop('score_dif_p2	server_p2	p1_ad p2_ad p1_break_points p2_break_points 	p1_break_points_won p2_break_points_won'.split(), axis=1)
        x = []# 特征数据
        y = []# 标签
        for index in df.index.values:
            # print(df.loc[index].values) 
            x.append(df.loc[index].values[:-2])
            y.append(df.loc[index].values[-2])
        x, y = np.array(x), np.array(y)
        x_list.append(x)
        y_list.append(y)
    return np.concatenate(x_list), np.concatenate(y_list).astype(np.int8)


In [6]:
X, Y = ds_concat()
Ymean, Ystd = Y.mean(), Y.std()
n = len(Y)
x, y = X[:300], Y[:300]
x_train, x_test = X[:-n//8], X[-n//8:]
y_train, y_test = Y[:-n//8], Y[-n//8:]

In [7]:
print(len(X[0]), len(x_test))

4 911


In [8]:
# # xgboost
# from xgboost import XGBClassifier
# xgbc_model=XGBClassifier()

# # 随机森林
# from sklearn.ensemble import RandomForestClassifier
# rfc_model=RandomForestClassifier()

# # ET
# from sklearn.ensemble import ExtraTreesClassifier
# et_model=ExtraTreesClassifier()

# # 朴素贝叶斯
# from sklearn.naive_bayes import GaussianNB
# gnb_model=GaussianNB()

# #K最近邻
# from sklearn.neighbors import KNeighborsClassifier
# knn_model=KNeighborsClassifier()

#逻辑回归
# from sklearn.linear_model import LogisticRegression
# lr_model=LogisticRegression()

# #决策树
# from sklearn.tree import DecisionTreeClassifier
# dt_model=DecisionTreeClassifier()

# #支持向量机
# from sklearn.svm import SVC
# svc_model=SVC()

# # xgboost
# xgbc_model.fit(x,y)

# # 随机森林
# rfc_model.fit(x,y)

# # ET
# et_model.fit(x,y)

# # 朴素贝叶斯
# gnb_model.fit(x,y)

# # K最近邻
# knn_model.fit(x,y)

# # 逻辑回归
# lr_model.fit(x,y)

# # 决策树
# dt_model.fit(x,y)

# # 支持向量机
# svc_model.fit(x,y)

# from sklearn.model_selection import cross_val_score
# print("\n使用５折交叉验证方法得随机森林模型的准确率（每次迭代的准确率的均值）：")
# print("\tXGBoost模型：",cross_val_score(xgbc_model,x,y,cv=5).mean())
# print("\t随机森林模型：",cross_val_score(rfc_model,x,y,cv=5).mean())
# print("\tET模型：",cross_val_score(et_model,x,y,cv=5).mean())
# print("\t高斯朴素贝叶斯模型：",cross_val_score(gnb_model,x,y,cv=5).mean())
# print("\tK最近邻模型：",cross_val_score(knn_model,x,y,cv=5).mean())
# print("\t逻辑回归：",cross_val_score(lr_model,x,y,cv=5).mean())
# print("\t决策树：",cross_val_score(dt_model,x,y,cv=5).mean())
# print("\t支持向量机：",cross_val_score(svc_model,x,y,cv=5).mean())

In [9]:
# 性能评估以XGboost为例
# xgb = XGBRegressor()
# # 对训练集训练模型
# xgb.fit(x_train, y_train)

# # 对测试集进行预测
# y_pred = xgb.predict(x_test)


In [10]:
# y_pred = y_pred
# y_test = y_test
# abs(y_pred - y_test).mean()

In [11]:
# plt.plot(y_test[:100], color='red')
# plt.plot(y_pred[:100], color='blue')

In [12]:
x_train[2]

array([-0.95307831,  1.        , -0.40824829, -0.39090011])

In [13]:
def model_adjust_parameters(cv_params, other_params):
    """模型调参"""
    # 模型基本参数
    model = XGBClassifier(**other_params)
    # sklearn提供的调参工具，训练集k折交叉验证
    optimized_param = GridSearchCV(estimator=model, param_grid=cv_params, scoring='neg_mean_absolute_error', cv=5, verbose=1)
    # 模型训练
    optimized_param.fit(x_train, y_train)
    # 对应参数的k折交叉验证平均得分
    means = optimized_param.cv_results_['mean_test_score']
    params = optimized_param.cv_results_['params']
    for mean, param in zip(means, params):
        print("mean_score: %f,  params: %r" % (mean, param))
    # 最佳模型参数
    print('参数的最佳取值：{0}'.format(optimized_param.best_params_))
    # 最佳参数模型得分
    print('最佳模型得分:{0}'.format(optimized_param.best_score_))

    # 模型参数调整得分变化曲线绘制
    parameters_score = pd.DataFrame(params, means)
    parameters_score['means_score'] = parameters_score.index
    parameters_score = parameters_score.reset_index(drop=True)
    # parameters_score.to_excel('parameters_score.xlsx', index=False)
    # 画图
    plt.figure(figsize=(15, 12))
    plt.subplot(2, 1, 1)
    plt.plot(parameters_score.iloc[:, :-1], 'o-')
    plt.legend(parameters_score.columns.to_list()[:-1], loc='upper left')
    plt.title('Parameters_size', loc='left', fontsize='xx-large', fontweight='heavy')
    plt.subplot(2, 1, 2)
    plt.plot(parameters_score.iloc[:, -1], 'r+-')
    plt.legend(parameters_score.columns.to_list()[-1:], loc='upper left')
    plt.title('Score', loc='left', fontsize='xx-large', fontweight='heavy')
    plt.show()
def xgboost_parameters():
    """模型调参过程"""
    # 第二步：min_child_weight 以及 max_depth
    # 参数的最佳取值：{'max_depth': 2, 'min_child_weight': 1}
    # 最佳模型得分:0.9180952380952381，模型分数未提高
    params = {
        'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        'min_child_weight': [1, 2, 3, 4, 5, 6],
        'gamma':[0, 0.03, 0.05, 0.08, 0.1, 0.3, 0.5, 0.7, 0.9, 1],
        'colsample_bytree':[0.3,0.5,0.6, 0.7, 0.8, 0.9, 1],
        "n_estimators": [50,60,70,80,90,100,120,150,200], # 多少棵树
        "eta": [0.05, 0.1, 0,2, 0.3], # 学习率
        }
	
	# 其他参数设置，每次调参将确定的参数加入，不写即默认参数
    fine_params = {'n_estimators': 50}
    return params, fine_params
    
adj_params, fixed_params = xgboost_parameters()
model_adjust_parameters(adj_params, fixed_params)

Fitting 5 folds for each of 189000 candidates, totalling 945000 fits


KeyboardInterrupt: 

In [ ]:
# 带上server标签时bst: n=188, m=10
params = {'objective':'multi:softprob',
    'n_estimators': 50, 'max_depth': 2, 'min_child_weight': 1, 'gamma': 0.1, 'colsample_bytree': 1,
                'subsample': 1, 'reg_alpha': 0.01, 'reg_lambda': 3, 'learning_rate': 0.4}
xgb= XGBClassifier(params)
xgb.fit(x_train, y_train)
print()

In [ ]:
st = -100
y_train_pred = y_pred = xgb.predict(x_train)
plt.plot(y_train[st:], color='red')
plt.plot(y_train_pred[st:], color='blue')

In [ ]:

y_pred = xgb.predict(x_test)
y_pred_prob = xgb.predict_proba(x_test)
y_pred_prob_mean = []
for j in y_pred_prob:
    cur = 0
    for i in range(6):
        cur += j[i]*i
    y_pred_prob_mean.append(cur)
y_pred_prob_mean = np.array(y_pred_prob_mean)

In [ ]:
start = 600
end = 700
plt.plot(y_test[start:end], color='red')
plt.plot(y_pred[start:end], color='green')
# plt.plot(y_pred_prob_mean[start:], color='blue')

In [ ]:
acc = 0
final_acc = 0
final_acc_win_ldx = []
final_acc_loss_ldx = []
for i in range(len(y_pred)):
    if y_pred[i]>2.5 and y_test[i]>2.5 or y_pred[i]<2.5 and y_test[i]<2.5:
        final_acc += 1
        if y_pred[i]>2.5 and y_test[i]>2.5: final_acc_win_ldx.append(i)
        if y_pred[i]<2.5 and y_test[i]<2.5: final_acc_loss_ldx.append(i)
    if y_pred[i]==y_test[i]:
        acc += 1
print("acc: {}".format(acc*100/len(y_pred)))
print("final_acc: {}".format(final_acc*100/len(y_pred)))

In [ ]:
plt.plot(y_test[start:end], color='blue')
xwplot = [i-start for i in final_acc_win_ldx if end > i >= start]
xlplot = [i-start for i in final_acc_loss_ldx if end > i >= start]
plt.scatter(xwplot, [2.5]*(len(xwplot)), color='green')
plt.scatter(xlplot, [2.5]*(len(xlplot)), color='red')

In [ ]:
def feature_importance_selected(clf_model):
    """模型特征重要性提取与保存"""
    # 模型特征重要性打印和保存
    feature_importance = clf_model.get_booster().get_fscore()
    feature_importance = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)
    # feature_ipt = pd.DataFrame(feature_importance, columns=['特征名称', '重要性'])
    # feature_ipt.to_csv('feature_importance.csv', index=False)
    print('特征重要性:', feature_importance)

    # 模型特征重要性绘图
    plot_importance(xgb)
    plt.show()

feature_importance_selected(xgb)

In [ ]:
from sklearn.metrics import precision_score, recall_score,roc_curve
def metrics_sklearn(y_valid, y_pred_):
    """模型对验证集和测试集结果的评分"""
    # 准确率
    accuracy = accuracy_score(y_valid, y_pred_)
    print('Accuracy：%.2f%%' % (accuracy * 100))

    # 精准率
    precision = precision_score(y_valid, y_pred_)
    print('Precision：%.2f%%' % (precision * 100))

    # 召回率
    recall = recall_score(y_valid, y_pred_)
    print('Recall：%.2f%%' % (recall * 100))

    # F1值
    f1 = f1_score(y_valid, y_pred_)
    print('F1：%.2f%%' % (f1 * 100))

    # auc曲线下面积
    auc = roc_auc_score(y_valid, y_pred_)
    print('AUC：%.2f%%' % (auc * 100))

    # ks值
    fpr, tpr, thresholds = roc_curve(y_valid, y_pred_)
    ks = max(abs(fpr - tpr))
    print('KS：%.2f%%' % (ks * 100))
metrics_sklearn(y_test,y_pred)